In [5]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import gdown
import shutil
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

/var/folders/f8/_srl45g93hb_f9bb62qtf1t80000gn/T/ipykernel_43367/1685725648.py:33: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [7]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[8933814000, 3600314002, 3952214000, 378523400..."
1,Name,object,14,0.00,779583,"[DESIGN CENTER, David & Brian Child Care, LLC,..."
2,City,object,30,0.00,32581,"[DEL MAR, SANTA FE, HARRISBURG, JACKSONVILLE, ..."
3,State,object,14,0.00,51,"[FL, TX, NV, WA, KS]"
4,Zip,int64,0,0.00,33611,"[84107, 98664, 89701, 14851, 77365]"
5,Bank,object,1559,0.17,5802,"[U.S. BANK NATIONAL ASSOCIATION, FIFTH THIRD B..."
6,BankState,object,1566,0.17,56,"[PA, CA, NY, IL]"
7,NAICS,int64,0,0.00,1312,"[532299, 445291, 238350, 722110, 0]"
8,ApprovalDate,object,0,0.00,9859,"[13-Sep-07, 24-Sep-07, 5-Aug-96, 17-Nov-99, 3-..."
9,ApprovalFY,object,0,0.00,70,"[2010, 2005, 2004, 2001, 2003]"


In [8]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [9]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [10]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)].astype(object)
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [11]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [12]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [13]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [14]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [15]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [16]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [17]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [18]:
df.shape

(675026, 13)

In [19]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[31-33, 54, 72, 62]"
1,Term,object,0,0.0,377,"[60, 24, 90, 180, 63]"
2,NoEmp,object,0,0.0,499,"[3, 2, 1, 6]"
3,NewExist,object,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,object,0,0.0,216,"[0, 25]"
5,RetainedJob,object,0,0.0,323,"[0, 2]"
6,UrbanRural,object,0,0.0,3,"[1, 0]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[200000, 15000, 10000, 30000]"


In [20]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [21]:
df = pd.read_csv(imported_csv_path)

/var/folders/f8/_srl45g93hb_f9bb62qtf1t80000gn/T/ipykernel_43367/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [22]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [23]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [24]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [25]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [26]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [27]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [28]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [29]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [30]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [31]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [32]:
df.shape

(460816, 13)

In [33]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[48-49, 62, 56, 54, 31-33]"
1,Term,int64,0,0.0,373,"[84, 36, 48, 19]"
2,NoEmp,int64,0,0.0,432,"[52, 3, 1, 17, 5]"
3,NewExist,object,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,[0]
5,RetainedJob,int64,0,0.0,276,"[3, 2, 0, 33, 4]"
6,UrbanRural,int64,0,0.0,3,"[1, 0, 2]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,"[N, Y]"
9,GrAppv,int64,0,0.0,12824,"[100000, 407700, 11800, 220000, 218000]"


In [34]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [35]:
df = pd.read_csv(imported_csv_path)

/var/folders/f8/_srl45g93hb_f9bb62qtf1t80000gn/T/ipykernel_43367/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [36]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [37]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [38]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [39]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [40]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [41]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [42]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [43]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [44]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [45]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [46]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [47]:
df.shape

(675026, 13)

In [48]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[54, 56, 72, 42]"
1,Term,int64,0,0.0,377,"[1, 18, 84, 300, 90]"
2,NoEmp,int64,0,0.0,499,"[10, 1, 7, 5]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 7]"
5,RetainedJob,int64,0,0.0,323,"[1, 0, 10, 3]"
6,UrbanRural,int64,0,0.0,3,"[0, 1]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,"[N, Y]"
9,GrAppv,int64,0,0.0,16880,"[80000, 25000, 35000, 250000, 72000]"


In [49]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv


In [50]:
df = pd.read_csv(imported_csv_path)

/var/folders/f8/_srl45g93hb_f9bb62qtf1t80000gn/T/ipykernel_43367/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [51]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[8348803000, 2123444008, 3143414001, 623752400..."
1,Name,object,14,0.00,779583,"[INTERSTATE SEALANT & CONCRETE, CHUNG EYE CARE..."
2,City,object,30,0.00,32581,"[ARLINGTON, HOUSTON, ZILLAH, HAYWARD, ORLANDO]"
3,State,object,14,0.00,51,"[AZ, AR, TX, GA]"
4,Zip,int64,0,0.00,33611,"[93101, 11210, 46234, 14209, 60153]"
5,Bank,object,1559,0.17,5802,"[BANK OF AMERICA NATL ASSOC, ZIONS FIRST NATIO..."
6,BankState,object,1566,0.17,56,"[NH, NC, VA, NY]"
7,NAICS,int64,0,0.00,1312,"[485999, 422690, 0, 238330]"
8,ApprovalDate,object,0,0.00,9859,"[16-Oct-07, 3-Dec-09, 28-Jan-91, 14-Dec-05, 1-..."
9,ApprovalFY,object,0,0.00,70,"[2004, 2006, 1993]"


In [52]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [53]:
df['NewExist'] = df['NewExist'].isin({1.0, 2.0}).astype(object)

In [54]:
is_valide = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide).astype(object)

In [55]:
is_valide = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide).astype(object)

In [56]:
df = df.dropna(subset=['NAICS'])
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [57]:
mapping = {0: 'unknown', 1: 'urban', 2: 'rural'}
df['UrbanRural'] = df['UrbanRural'].map(mapping).astype(object)

In [58]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1).astype(object)
df = df.drop('FranchiseCode', axis=1)

In [59]:
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y').dt.strftime('%d-%m-%y').astype(object)

In [60]:
df['ApprovalDate'].unique()

array(['28-02-97', '07-02-06', '25-03-97', ..., '25-02-97', '26-02-97',
       '27-02-97'], dtype=object)

In [61]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:]).astype(object)
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [62]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [63]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [64]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [65]:
df['MIS_Status'].value_counts()

MIS_Status
P I F     550334
CHGOFF    145335
Name: count, dtype: int64

In [66]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0).astype(bool)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [67]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [68]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,Name,object,0,0.0,611131,"[Dan P. Nutton dba Dan P. Nutt, J A G ROOFING..."
1,City,object,0,0.0,28860,"[Tampa, OXON HILL, COLUMBUS, CLEVELAND, LEBANON]"
2,State,object,0,0.0,51,"[WA, CA, NM, VA, RI]"
3,Bank,object,0,0.0,5221,"[COMMUNITY BANK & TRUST, JPMORGAN CHASE BANK N..."
4,BankState,object,0,0.0,55,"[RI, VA, NC, OR]"
5,NAICS,object,0,0.0,20,"[31-33, 62, 81, 72]"
6,ApprovalDate,object,0,0.0,7367,"[02-11-04, 02-08-04, 17-11-00, 30-12-03, 10-03..."
7,ApprovalFY,object,0,0.0,29,"[2006, 1994, 2005, 2008]"
8,Term,int64,0,0.0,378,"[84, 69, 24]"
9,NoEmp,int64,0,0.0,502,"[1, 6, 9, 40]"


In [69]:
df.shape

(696572, 20)

In [70]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_Pure.csv
